# 早期終了 (Early stopping)

In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.2の大きさのノイズが乗ったデータを考えます。データ数は30にします。

In [ ]:
np.random.seed(0)

nSample = 30

x = np.linspace(-np.pi, np.pi, nSample)
t0 = np.sin(x)
t = t0 + 0.2 * np.random.randn(nSample)

# プロット
plt.scatter(x, t, s=10, c='black', label='data')  # データ点のプロット
x_grid = np.linspace(-np.pi, np.pi, 100)
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、隠れ層が5層、ノード数が128の隠れ層を4層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential([
    Dense(128, activation='relu', input_dim=1),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が128の層を追加。活性化関数はReLU関数。
    Dense(1, activation='linear'),  # ノード数が1の層を追加。活性化関数は線形関数。
])

#　誤差関数として二乗和誤差、最適化関数としてadamを使用
model.compile(loss='mean_squared_error', optimizer='adam')

#  トレーニング
model.fit(
    x=x, 
    y=t,
    epochs=1000,
    verbose=0
)

# プロット
plt.scatter(x, t, s=10, c='black', label='data')  # データ点のプロット
# x_grid = np.linspace(-np.pi, np.pi, 100)
plt.plot(x_grid, model.predict(x_grid), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()


モデルの出力(赤線)がトレーニングデータ点に強くフィットしてしまっていることがわかります。

## データの分割
Early stoppingでは データセットをランダムに2つに分割します。
一方のデータセット(training dataset)でトレーニングをし、もう一方のデータセット(validation dataset)で過学習がおこらないかを確認します。
サンプルの分割は手で行うこともできますが、scikit learningというライブラリに便利な関数が用意されているので、これを用いることにします。

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.2)

上の例では、全体の80%をトレーニングデータ、全体の20%を検証用データ(validation sample)として分割しています。

検証用データに対して監視をするには、`fit`関数の中で、検証用データを与えます。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential([
    Dense(128, activation='relu', input_dim=1),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(1, activation='linear'),  # ノード数が1の層を追加。活性化関数は線形関数。
])

model.compile(loss='mean_squared_error', optimizer='adam')

# トレーニング
history = model.fit(
    x=x_train,
    y=t_train,
    validation_data=(x_valid, t_valid),
    epochs=300,
    verbose=0
)

# ロス関数の推移を取得します
loss_train = history.history['loss']
loss_valid = history.history['val_loss']


結果をプロットしてみます。

In [ ]:
# データ点とモデル予測値のプロット
plt.scatter(x_train, t_train, s=10, c='black', label='training data')  # データ点のプロット
plt.scatter(x_valid, t_valid, s=10, c='gray', label='validation data')  # データ点のプロット
plt.plot(x_grid, model.predict(x_grid), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(x)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train, label='loss (train)')
plt.plot(loss_valid, label='loss (valid)')
plt.legend()
plt.show()

学習に使った黒点に沿うようにモデルがフィットされていることがわかります。
一方で、検証用の点(灰色の点)に対しての予測値は大きくずれてしまっています。

誤差関数の値(loos)の推移を見ると、epoch数が およそ100 あたりで、検証用データの誤差関数の値が下がらなくなってしまってしまい、学習を進めるにつれ、検証用データに対する誤差はむしろ大きくなってしまっています。
これが過学習を起こした状態で、トレーニングデータに強く学習してしまったため、本来の目的である汎化誤差が大きくなってしまっています。

これは、検証用データの誤差関数値がそれ以上下がらなくなった点で学習を止めることで緩和することができます。
上の例だとおよそ100 エポックぐらいで誤差関数値が上昇に転じているので、このあたりで学習を止めることにしましょう。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential([
    Dense(128, activation='relu', input_dim=1),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(1, activation='linear'),  # ノード数が1の層を追加。活性化関数は線形関数。
])

model.compile(loss='mean_squared_error', optimizer='adam')

# トレーニング
history = model.fit(
    x=x_train,
    y=t_train,
    validation_data=(x_valid, t_valid),
    epochs=100,
    verbose=0
)

# ロス関数の推移を取得します
loss_train = history.history['loss']
loss_valid = history.history['val_loss']

# データ点とモデル予測値のプロット
plt.scatter(x_train, t_train, s=10, c='black', label='training data')  # データ点のプロット
plt.scatter(x_valid, t_valid, s=10, c='gray', label='validation data')  # データ点のプロット
plt.plot(x_grid, model.predict(x_grid), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(x)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train, label='loss (train)')
plt.plot(loss_valid, label='loss (valid)')
plt.legend()
plt.show()

今度はどうなったでしょうか？おそらく、過学習が大幅に抑制されているのではないかと思います。

このように、学習を途中で打ち切るような手法を早期終了(Early Stopping)と言います。
Keras/Tensorflowでは、これが簡単に行えるようにコールバックが用意されています。

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

(他にもいろいろなコールバックが用意されています。コールバックは、学習の前後や学習の途中で追加で何か処理をしたい時に使います。)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=20, restore_best_weights=True)

`patience`というのは、モニターしたい値(ここでは検証用データのロス)が何エポック下がらなくなったら学習を止めるか、を指定します。今はちょっと大きめの値の20としておきます。
この値は問題・モデルによって調節するべき値です。小さすぎると、過学習を起こす前・学習が十分に行われていない段階で学習がストップしてしまいます。一方で、大きすぎると学習時間が余計にかかってしまいます。

`restore_best_weights`がFalseだと、early stoppingが終わったときのモデルの重みがそのまま残ります。そのため、`restore_best_weights`がFalseかつ`patience`を大きくすると、過学習が起こってしまった値が記録されてしまいます。この値をTrueにしておくと、ロスが最小になったときのモデルの重みでモデルがアップデートされます。

このコールバックを`fit` の引数として与えることで、Early stoppingが行われます。

In [ ]:
# モデルの定義
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential([
    Dense(128, activation='relu', input_dim=1),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(128, activation='relu'),  # ノード数が64の層を追加。活性化関数はReLU関数。
    Dense(1, activation='linear'),  # ノード数が1の層を追加。活性化関数は線形関数。
])

model.compile(loss='mean_squared_error', optimizer='adam')

# 早期終了(Early stopping)コールバック
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=20, restore_best_weights=True)

# トレーニング
history = model.fit(
    x=x_train,
    y=t_train,
    validation_data=(x_valid, t_valid),
    epochs=500,
    callbacks=es,  # ここでコールバックを指定します。
    verbose=0
)

print(f"stopped epoch = {es.stopped_epoch}, patience = {es.patience}, best epoch = {es.stopped_epoch - es.patience}")

# ロス関数の推移を取得します
loss_train = history.history['loss']
loss_valid = history.history['val_loss']

# データ点とモデル予測値のプロット
plt.scatter(x_train, t_train, s=10, c='black', label='training data')  # データ点のプロット
plt.scatter(x_valid, t_valid, s=10, c='gray', label='validation data')  # データ点のプロット
plt.plot(x_grid, model.predict(x_grid), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(x)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train, label='loss (train)')
plt.plot(loss_valid, label='loss (valid)')
plt.legend()
plt.show()

指定した最大エポック数(500エポック)の前に学習が終了していると思います。また、過学習も緩和されています。